# 2.4 Histograms Tutorial
By Pac Hung and Zac Todd

The image used in this tutoial is large so examples maybe take a couple of secounds to run.

In [ ]:
import os
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt


# Increase output size of images viewer
plt.rcParams['figure.figsize'] = [19, 10]


IMAGES_DIR = f"{os.getcwd()}/resources"

In [ ]:
img = plt.imread(f"{IMAGES_DIR}/histogram.jpg")
plt.imshow(img)
plt.show()

Below is the histogram of the image in grey scale.

In [ ]:
def draw_gray_hist(img, title='original'):
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    h, w = gray_img.shape[:2]
    pixelSequence = gray_img.reshape([h * w, ])
    numberBins = 256
    histogram, bins, patch = plt.hist(pixelSequence, numberBins,
                                      facecolor='black', histtype='bar')
    plt.xlabel("gray label")
    plt.ylabel("number of pixels")
    plt.title(title)
    plt.axis([0, 255, 0, np.max(histogram)])
    plt.show()
    

draw_gray_hist(img)

In [ ]:
def histogram_transform(img, transform_func):
    '''
    Implement histogram transformation for each color channel of a image
    :param img: 3 channel image
    :param transform_func: histogram transformation function to be used
    :return: image after histogram transformation
    '''
    # implement histogram transformation for each color channel
    r = img[:, :, 0]
    g = img[:, :, 1]
    b = img[:, :, 2]
    
    r_out = transform_func(r)
    g_out = transform_func(g)
    b_out = transform_func(b)
    
    equalized_out = np.stack((r_out, g_out, b_out), axis=-1)
    draw_gray_hist(equalized_out, 'histogram_transformed')
    plt.imshow('histogram_transformed', equalized_out)
    plt.show()


def normalize_transform(img):
    '''
    Normalize the histogram of a grayscale image(or any one channel images)
    :param img: grayscale image(or any one channel images)
    :return: image after normalize transformation
    '''
    # find the global minimun and maximun of image array
    in_min, in_max = cv2.minMaxLoc(img)[:2] 
    out_min, out_max = 0, 255
    
    # calculate a and b and output = a * img + b
    a = float(out_max - out_min) / (in_max - in_min)
    b = out_min - a * out_min
    output = a * img + b
    output = output.astype(np.uint8)
    return output

histogram_transform(img, normalize_transform)

In [ ]:
def linear_transform(img, a=2.0, b=10.0):
    '''
    :param img: grayscale image(or any one channel images)
    :param a:  float  
    :param b:  float
    :return: output = a * img + b
    '''
    output = a * img + b
    output[output > 255] = 255
    output = np.around(output)
    output = output.astype(np.uint8)
    return output

histogram_transform(img, cv2.linear_transform)

In [ ]:
histogram_transform(img, cv2.equalizeHist)

In [ ]:
def equalize_transfrom_2(img):
    '''
    Equalize the histogram of a grayscale image(or any one channel images).
    The probability density function of the output images's histogram follows the discrete uniform distribution
    :param img: grayscale image(or any one channel images)
    :return: image after histogram equalization transformation
    '''
    h, w = img.shape
    # calculate the image's color histogram array
    gray_hist = np.zeros([256], np.uint64)
    for i in range(h):
        for j in range(w):
            gray_hist[img[i][j]] += 1

    # calculate the color histogram's discrete cumulative distribution
    cumulate_hist = np.zeros([256], np.uint32)
    for p in range(256):
        if p == 0:
            cumulate_hist[p] = gray_hist[0]
        else:
            cumulate_hist[p] = cumulate_hist[p - 1] + gray_hist[p]

    # calculate the histogram value mapping based on the (3-15) equation 
    # of 'histogram eaualization.pdf'
    output_hist_map = np.zeros([256], np.uint8)
    cofficient = 255.0 / (h * w)
    for p in range(256):
        q = cofficient * float(cumulate_hist[p]) - 1
        if q >= 0:
            output_hist_map[p] = math.floor(q)
        else:
            output_hist_map[p] = 0

    # project each pixel value of original image to the equalized histogram map
    equalized_hist_img = np.zeros(img.shape, np.uint8)
    for i in range(h):
        for j in range(w):
            equalized_hist_img[i][j] = output_hist_map[img[i][j]]
    return equalized_hist_img

histogram_transform(img, equalize_transfrom_2)